In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import mpl_toolkits.mplot3d as a3
import pandas as pd

In [8]:
class Rivelatori:
    def __init__(self, a_t1, b_t1, a_t2, b_t2, h_t, a_m, b_m, h_m, detector_height, total_time_hours, density, muon_energy, light_yield):
        self.a_t1 = a_t1 #metri
        self.b_t1 = b_t1
        self.a_t2 = a_t2 #metri
        self.b_t2 = b_t2
        self.h_t = h_t
        self.a_m = a_m
        self.b_m = b_m
        self.h_m = h_m
        self.detector_height = detector_height
        self.total_time_hours = total_time_hours
        self.density = density #g /cm^3
        self.muon_energy = muon_energy #MeV cm^2 / g
        self.light_yield = light_yield #ph / MeV

        self.x0s = None #piano trigger down
        self.y0s = None
        self.z0s = None
        self.x1s = None #piano trigger up
        self.y1s = None
        self.z1s = None
        self.xd = None #piano rivelatore down
        self.yd = None
        self.zd = None
        self.xu = None #piano rivelatore up
        self.yu = None
        self.zu = None
        self.x1v = None
        self.y1v = None
        self.z1v = None
        self.x2v = None
        self.y2v = None
        self.z2v = None
        
        
        self.us = None
        self.vs = None
        self.ws = None
        self.hitmax_trigger = None
        self.hitmax_measure_down = None
        self.hitmax_measure_up = None
        self.hitmax_measure_vert1 = None
        self.hitmax_measure_vert2 = None
        self.hitmax_measure = None
        self.num_trigger = None
        self.num_measure = None
        self.n = None
        self.S_t = None
        self.S_m = None
        self.expected_muons = None
        self.expected_muons_min = None
        self.released_energy = None
        self.photons = None
        self.muons_tot = None
        #self.d_m = None
        self.hitmax_measure1_ = None
        self.hitmax_measure2_ = None
        self.hitmax_measure3_ = None
        self.hitmax_measure4_ = None
        self.hitmax_measure5_ = None
        self.d1 = None
        self.d2 = None
        self.d3 = None
        self.d4 = None
        self.d5 = None
       

    def generate_rays(self):
        self.S_t = self.a_t1 * self.b_t1
        self.S_m = self.a_m * self.b_m
        self.expected_muons = self.S_t * 70 * 2 *np.pi
        self.expected_muons_min = self.expected_muons * 60
        self.n = int(self.expected_muons_min) * self.total_time_hours * 60 
        th = np.arccos((1 - np.random.random(self.n)) ** (1/3))
        ph = 2 * np.pi * np.random.random(self.n)
        st, ct = np.sin(th), np.cos(th)
        sp, cp = np.sin(ph), np.cos(ph)
        self.us, self.vs, self.ws = st * cp, st * sp, ct

        self.x0s = np.random.rand(self.n) * self.a_t1 - self.a_t1 / 2
        self.y0s = np.random.rand(self.n) * self.b_t1 - self.b_t1 / 2
        self.z0s = np.zeros(self.n)

        self.x1s = self.x0s + self.h_t * self.us/self.ws 
        self.y1s = self.y0s + self.h_t * self.vs/self.ws
        self.z1s = np.full(self.n, self.h_t)
        
        self.xd = self.x0s + self.detector_height * self.us/self.ws
        self.yd = self.y0s + self.detector_height * self.vs/self.ws
        self.zd = np.full(self.n, self.detector_height)
        
        self.xu = self.x0s + (self.detector_height + self.h_m) * self.us/self.ws
        self.yu = self.y0s + (self.detector_height + self.h_m) * self.vs/self.ws
        self.zu = np.full(self.n,(self.detector_height + self.h_m))
        
       

        
        
        self.x1v = self.x0s + (self.b_m/2 - self.y0s) * (self.us / self.vs)
        self.y1v = np.full(self.n, self.b_m/2)
        self.z1v = self.z0s + (self.b_m/2 - self.y0s) * (self.ws / self.vs)
        
        self.x2v = self.x0s + (-self.b_m/2 - self.y0s) * (self.us / self.vs)
        self.y2v = np.full(self.n, -self.b_m/2)
        self.z2v = self.z0s + (-self.b_m/2 - self.y0s) * (self.ws / self.vs)
        

        self.hitmax_trigger = (-self.a_t2/2 < self.x1s) & (self.x1s < self.a_t2/2) & (-self.b_t2/2 < self.y1s) & (self.y1s < self.b_t2/2)
        self.hitmax_measure_down = (-self.a_m/2 < self.xd) & (self.xd < self.a_m/2) & (-self.b_m/2 < self.yd) & (self.yd < self.b_m/2) 
        self.hitmax_measure_up = (-self.a_m/2 < self.xu) & (self.xu < self.a_m/2) & (-self.b_m/2 < self.yu) & (self.yu < self.b_m/2) 
        self.hitmax_measure_vert1 =  (-self.a_m/2 < self.x1v) & (self.x1v < self.a_m/2) & (self.detector_height < self.z1v) & (self.z1v < self.detector_height + self.h_m)
        self.hitmax_measure_vert2 =  (-self.a_m/2 < self.x2v) & (self.x2v < self.a_m/2) & (self.detector_height < self.z2v) & (self.z2v < self.detector_height + self.h_m)
         
        
        self.hitmax_measure1 = self.hitmax_trigger & self.hitmax_measure_up & self.hitmax_measure_down    
        self.hitmax_measure2 = self.hitmax_trigger & self.hitmax_measure_up & self.hitmax_measure_vert1
        self.hitmax_measure3 = self.hitmax_trigger & self.hitmax_measure_up & self.hitmax_measure_vert2   
        self.hitmax_measure4 = self.hitmax_trigger & self.hitmax_measure_down & self.hitmax_measure_vert1
        self.hitmax_measure5 = self.hitmax_trigger & self.hitmax_measure_down & self.hitmax_measure_vert2
        
        self.hitmax_measure = self.hitmax_measure1 + self.hitmax_measure2 + self.hitmax_measure3 + self.hitmax_measure4 + self.hitmax_measure5 
    
        self.num_trigger = np.count_nonzero(self.hitmax_trigger)
        self.num_measure = np.count_nonzero(self.hitmax_measure)
        self.num_1 = np.count_nonzero(self.hitmax_measure1)
        self.num_2 = np.count_nonzero(self.hitmax_measure2)
        self.num_3 = np.count_nonzero(self.hitmax_measure3)
        self.num_4 = np.count_nonzero(self.hitmax_measure4)
        self.num_5 = np.count_nonzero(self.hitmax_measure5)
    
    
        return self.num_trigger, self.num_measure, self.num_1, self.num_2, self.num_3, self.num_4, self.num_5
    
    def count_photons(self):
    

        self.generate_rays()
        self.d1 = np.sqrt( (self.xd - self.xu )**2 + ( self.yd - self.yu )**2 + ( self.zd - self.zu )**2 )
        self.d2 = np.sqrt( (self.x1v - self.xu )**2 + ( self.y1v - self.yu )**2 + ( self.z1v - self.zu )**2 )
        self.d3 = np.sqrt( (self.x2v - self.xu )**2 + ( self.y2v - self.yu )**2 + ( self.z2v - self.zu )**2 )
        self.d4 = np.sqrt( (self.xd - self.x1v )**2 + ( self.yd - self.y1v )**2 + ( self.zd - self.z1v )**2 )
        self.d5 = np.sqrt( (self.xd - self.x2v )**2 + ( self.yd - self.y2v )**2 + ( self.zd - self.z2v )**2 )
        #self.d_m = np.sqrt( (self.xd - self.xu )**2 + ( self.yd - self.yu )**2 + ( self.zd - self.zu )**2 )
        
        

        
        self.d_m = self.hitmax_trigger*(self.hitmax_measure1 * self.d1 + self.hitmax_measure2 * self.d2 + self.hitmax_measure3 * self.d3 + self.hitmax_measure4 * self.d4 + self.hitmax_measure5 * self.d5)

        
        
        self.released_energy = self.d_m * self.density * self.muon_energy
        self.photons = self.released_energy * self.light_yield
        self.muons_tot = self.total_time_hours * self.expected_muons_min *  60  #cm
        
        
        muon_data = []
        for i in range(int(self.num_measure)):          
            
            muon_entry = {
                'length_traveled': self.d_m[i],
                'released_energy': self.released_energy[i],
                'photons': self.photons[i],
                'trigger' : self.hitmax_trigger[i],
                'measure': self.hitmax_measure[i]
            }
            muon_data.append(muon_entry)

        muon_df = pd.DataFrame(muon_data)

        return self.released_energy, self.photons, self.d_m,  muon_df

    
        

    def plot_rays(self, graph_rays=False, graph_trigger=False, graph_measure=False):
        self.generate_rays()

        if graph_rays:
            fig = plt.figure(figsize=(6, 6), dpi=100)
            ax = fig.add_subplot(111, projection='3d')
            ax.view_init(20, 20)
            nmax = min(self.n, 100)
            ax.quiver(self.x0s[:nmax], self.y0s[:nmax], self.z0s[:nmax], self.us[:nmax], self.vs[:nmax], self.ws[:nmax], length=1, arrow_length_ratio=0, alpha=0.4)
            ax.set_xlim3d(-0.1, 0.1)
            ax.set_ylim3d(-0.1, 0.1)
            ax.set_zlim3d(0, 0.1)
            

            plt.savefig("Figures/plot_rays.png")

    def plot_trigger(self, graph_trigger=False):
        self.generate_rays()

        if graph_trigger:
            fig = plt.figure(figsize=(6, 6), dpi=100)
            ax = fig.add_subplot(111, projection='3d')
            ax.view_init(20, 20)
            nmax = min(self.n, 100)
            for i in range(nmax):
                if self.hitmax_trigger[i]:
                    ax.plot([self.x0s[i], self.x1s[i]], [self.y0s[i], self.y1s[i]], [self.z0s[i], self.z1s[i]], color='red', alpha=0.4)

            ax.set_xlim3d(-0.1, 0.1)
            ax.set_ylim3d(-0.1, 0.1)
            ax.set_zlim3d(0, 0.1)

            plane_vtx1 = np.array([[- self.a_t1/2, - self.b_t1/2, 0], [self.a_t1/2, -self.b_t1/2, 0], [self.a_t1/2, self.b_t1/2, 0], [-self.a_t1/2, self.b_t1/2, 0], [-self.a_t1/2, -self.b_t1/2, 0]])
            plane0 = a3.art3d.Poly3DCollection([plane_vtx1])
            plane0.set_color('#0000ff20')
            plane0.set_edgecolor('#0000ff')
            ax.add_collection3d(plane0)

            plane_vtx2 = np.array([[- self.a_t2/2, - self.b_t2/2, self.h_t], [self.a_t2/2, -self.b_t2/2, self.h_t], [self.a_t2/2, self.b_t2/2, self.h_t], [-self.a_t2/2, self.b_t2/2, self.h_t], [-self.a_t2/2, -self.b_t2/2, self.h_t]])
            plane1 = a3.art3d.Poly3DCollection([plane_vtx2])
            plane1.set_color('#0000ff20')
            plane1.set_edgecolor('#0000ff')
            ax.add_collection3d(plane1)
            
         

        

            plt.savefig("Figures/plot_trigger.png")

    def plot_measure(self, graph_measure=False):
        self.generate_rays()

        if graph_measure:

            hitmax_measure_indices = np.where(self.hitmax_measure)[0]  
            nmax = min(len(hitmax_measure_indices), 100)  
            hitmax_measure_indices = hitmax_measure_indices[:nmax]   
            
            
            fig = plt.figure(figsize=(6, 6), dpi=100)
            ax = fig.add_subplot(111, projection='3d')
            ax.view_init(20, 20)
            
            

            ax.quiver(self.x0s[hitmax_measure_indices], self.y0s[hitmax_measure_indices], self.z0s[hitmax_measure_indices], self.us[hitmax_measure_indices], self.vs[hitmax_measure_indices], self.ws[hitmax_measure_indices], length=2, arrow_length_ratio=0, color='darkgreen', alpha=0.4)
            ax.scatter(self.x1s[hitmax_measure_indices], self.y1s[hitmax_measure_indices], self.z1s[hitmax_measure_indices], s=4, color='darkgreen')
            ax.set_xlim3d(-0.1, 0.1)
            ax.set_ylim3d(-0.1, 0.1)
            ax.set_zlim3d(0, 0.1)

            plane_vtx1 = np.array([[- self.a_t1/2, - self.b_t1/2, 0], [self.a_t1/2, -self.b_t1/2, 0], [self.a_t1/2, self.b_t1/2, 0], [-self.a_t1/2, self.b_t1/2, 0], [-self.a_t1/2, -self.b_t1/2, 0]])
            plane0 = a3.art3d.Poly3DCollection([plane_vtx1])
            plane0.set_color('#0000ff20')
            plane0.set_edgecolor('#0000ff')
            ax.add_collection3d(plane0)

            plane_vtx2 = np.array([[- self.a_t2/2, - self.b_t2/2, self.h_t], [self.a_t2/2, -self.b_t2/2, self.h_t], [self.a_t2/2, self.b_t2/2, self.h_t], [-self.a_t2/2, self.b_t2/2, self.h_t], [-self.a_t2/2, -self.b_t2/2, self.h_t]])
            plane1 = a3.art3d.Poly3DCollection([plane_vtx2])
            plane1.set_color('#0000ff20')
            plane1.set_edgecolor('#0000ff')
            ax.add_collection3d(plane1)

            third_plane_vtx1 = np.array([[-self.a_m/2, -self.b_m/2, self.detector_height], [self.a_m/2, -self.b_m/2, self.detector_height], [self.a_m/2, self.b_m/2, self.detector_height], [-self.a_m/2, self.b_m/2, self.detector_height], [-self.a_m/2, -self.b_m/2, self.detector_height]])
            third_plane_vtx2 = np.array([[-self.a_m/2, -self.b_m/2, self.detector_height+self.h_m], [self.a_m/2, -self.b_m/2, self.detector_height+self.h_m], [self.a_m/2, self.b_m/2, self.detector_height+self.h_m], [-self.a_m/2, self.b_m/2, self.detector_height+self.h_m], [-self.a_m/2, -self.b_m/2, self.detector_height+self.h_m]])

            third_plane1 = a3.art3d.Poly3DCollection([third_plane_vtx1])
            third_plane2 = a3.art3d.Poly3DCollection([third_plane_vtx2])
            third_plane1.set_color('#00ff0020')  
            third_plane1.set_edgecolor('#00ff00')  
            third_plane2.set_color('#00ff0020')  
            third_plane2.set_edgecolor('#00ff00')  
            ax.add_collection3d(third_plane1)
            ax.add_collection3d(third_plane2)
            
            vert_plane1_vtx = np.array([
            [self.a_m/2, -self.b_m/2, self.detector_height],
            [self.a_m/2, -self.b_m/2, self.detector_height + self.h_m],
            [self.a_m/2, self.b_m/2, self.detector_height + self.h_m],
            [self.a_m/2, self.b_m/2, self.detector_height]
            ])
            vert_plane2_vtx = np.array([
            [-self.a_m/2, -self.b_m/2, self.detector_height],
            [-self.a_m/2, -self.b_m/2, self.detector_height + self.h_m],
            [-self.a_m/2, self.b_m/2, self.detector_height + self.h_m],
            [-self.a_m/2, self.b_m/2, self.detector_height]
            ])
            vert_plane1 = a3.art3d.Poly3DCollection([vert_plane1_vtx])
            vert_plane2 = a3.art3d.Poly3DCollection([vert_plane2_vtx])
            vert_plane1.set_color('#ff000020')
            vert_plane1.set_edgecolor('#ff0000')
            vert_plane2.set_color('#ff000020')
            vert_plane2.set_edgecolor('#ff0000')
            ax.add_collection3d(vert_plane1)
            ax.add_collection3d(vert_plane2)
            
            plt.savefig("Figures/plot_measure.png")

            plt.show()
            
        return('Numero totale dei raggi', self.n,
               'Numero di intersezioni rivelatori di trigger: ', self.num_trigger, 
               'Numero di intersezioni rivelatori di misura: ', self.num_measure,
               'Percentuale trigger/tot', self.num_trigger/self.n *100,
               'Percentuale measure/tot', self.num_measure/self.n *100,
               'Accettanza geometrica', self.num_measure/self.num_trigger * 100,
               ' % Numero 1: ', self.num_1/self.num_measure * 100,
               ' % Numero 2: ', self.num_2/self.num_measure * 100,
               ' % Numero 3: ', self.num_3/self.num_measure * 100,
               ' % Numero 4: ', self.num_4/self.num_measure * 100,
               ' % Numero 5: ', self.num_5/self.num_measure * 100,
               'Energia rilasciata per muone', self.released_energy, 'MeV',
               'Lunghezza percorsa', self.d_m,
               'Fotoni attesi per muone', self.photons,
              'Primi 10 dati', ((muon_df.query('measure'))['photons']).head(10),)

    def run(self, total_time_hours, get_dataframe=False, plot_graphs=False, plot_log_histogram=False):
        self.total_time_hours = total_time_hours
        self.generate_rays()

        if get_dataframe:
            released_energy, photons, d_m, muon_df = self.count_photons()

            if plot_graphs:
                plt.figure(figsize=(8, 6))
                plt.hist((muon_df.query('trigger'))['photons'], bins=20, color='blue', alpha=0.7)
                frequenze, bordi_bin = np.histogram((muon_df.query('trigger'))['photons'], bins = 20)
                valori_frequenti = []
                for _ in range(2):  
                    valore_piu_frequente = bordi_bin[np.argmax(frequenze)]
                    valori_frequenti.append(valore_piu_frequente)
                    frequenze = np.delete(frequenze, np.argmax(frequenze))
                    bordi_bin = np.delete(bordi_bin, np.argmax(frequenze))

                
                
                print("I primi due valori più frequenti sono:", valori_frequenti)
                #print("Valore piu probabile: ", valore_piu_probabile)
                plt.xlabel('Photons')
                plt.ylabel('Frequency')
                plt.title('Histogram of Photons')
                plt.grid(True)
                plt.savefig("Figures/histogram_photons.png")
                plt.show()
                
            if plot_log_histogram:
                plt.figure(figsize=(8, 6))
                plt.hist((muon_df.query('trigger'))['photons'], bins=20, color='green', alpha=0.7, log=True)
                plt.xlabel('Photons')
                plt.ylabel('Frequency')
                plt.title('Logarithmic Histogram of Photons')
                plt.grid(True)
                plt.savefig("Figures/log_histogram_photons.png")
                plt.show()

            return released_energy, photons,d_m,  muon_df
        else:
            self.generate_rays()

            if plot_graphs:
                self.plot_rays(graph_rays=True)
                self.plot_trigger(graph_trigger=True)
                self.plot_measure(graph_measure=True)
                plt.show()
            return self.generate_rays()
        
        
             